In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wg
from ipywidgets import Layout

#explaining the concept to the user
print('How does it work?\n\n1. Choose the league you want to test this method with.\n2. Each game we\'re betting\
 on the favortie team. But as we\'ve seen in the Excel file, not all odds categories have the same prediction accuracy.\
 Choose one among the four.\n3. Set your initial bankroll, and the amount of your initial stake. The multiplier will be used\
 after a lost bet to determine your next stake. We\'re going to multiply automatically your last stake until you win, so you can cover your losses\
 and be in profit.\nYou start over with the initial stake when you win a bet, or when\
 you don\'t have enough in your bankroll to multiply your last stake after a loss. If you don\'t have enough\
 to even make the first bet with the initial stake, you\'re out.')

#dataframe with all 10y-results in entry, returns a new dataframe sorted depending on the odds category 
#and the league selected by the user
def sort_category (odds_category, file_source, league):
    file = file_source[(file_source['Odds Category'] == odds_category) & (file_source['League'] == league) ]
    return file 

#plot the final graph
def plot_graph (a, b):
    plt.figure(figsize=(12,8))
    plt.plot(a,b,label='bankroll')
    plt.grid( color='grey', alpha=0.6, linestyle='dashdot')
    plt.xlabel('Date', fontsize =15)
    plt.ylabel('Money', fontsize =15)
    plt.legend()
    plt.title('When you always bet on the favourite team\n', fontsize =20)
    plt.show()

#set the 5 widgets to interact with the main function
style={'description_width': 'initial'}
bankroll_init = wg.IntSlider(min=1000, max=10000, step=500, value=3000, description='Initial Bankroll',style=style)
stake_init = wg.IntSlider(min=10, max=50, step=5, description='Initial Stake',style=style)
multiplier  = wg.IntSlider(min=2, max=10, step=1,description='Multiplier',style=style)
odds_category = wg.RadioButtons(options=[["< @1.5",1],["@1.5 - @2",2],["@2 - @2.5",3],["> @2.5",4]], 
                                description="Odds of favourite teams",style=style)                               
league = wg.Dropdown(options=['Ligue 1','Premiere League','Bundesliga','Serie A','La Liga'],
                     description='Choose your league:',style=style)

def main(bankroll_init, stake_init, multiplier,odds_category,league):
    
    #create a dataframe from the excel file that has all 10y-results
    file_source = pd.read_excel('Beat_the_Bookies.xlsx')
    #sort the dataframe 
    sorted_file = sort_category(odds_category,file_source, league)
    
    str_category = ['','under @1.5','between @1.5 et @2','between @2 et @2.5','over @2.5']
    str_bankrupcy='Congratulations!! You\'ve made it through these 10 seasons without loosing your initial bankroll!!'
    bankroll_current = bankroll_init
    stake_current = stake_init
    list_bankroll = []
    list_date = []
    
    #for loop through the sorted dataframe
    for i in sorted_file.index:
        
        #list all the dates and the evolution of the bankroll in 2 separated lists 
        #(futur x-axis and y-axis for the graph)
        list_bankroll.append(bankroll_current)
        list_date.append(sorted_file['Date'][i])
        
        #if enough money in the bankroll to make the bet
        if stake_current < bankroll_current:

            bankroll_current = bankroll_current - stake_current
            
            #if the home team was favourite and won
            if (sorted_file['B365H'][i] == sorted_file['Min Odds'][i]) & (sorted_file['FTR'][i] == 'H'):
                bankroll_current = bankroll_current + (stake_current * sorted_file['B365H'][i])
                stake_current = stake_init
            #if the away team was favourite and won
            elif (sorted_file['B365A'][i] == sorted_file['Min Odds'][i]) & (sorted_file['FTR'][i] == 'A'):
                bankroll_current = bankroll_current + (stake_current * sorted_file['B365A'][i])
                stake_current = stake_init
            #if the favourite team didn't win
            else :
                stake_current = stake_current * multiplier
        
        #if not enough money to even make the first bet with the initial stake -> Game Over
        elif bankroll_current < stake_init:
            str_bankrupcy = ('Sorry, not enough money in your bankroll to keep going... You have to quit the game early with '
                             +"{:.2f}".format(round(bankroll_current, 2))+'€, on the: '+sorted_file['Date'][i].strftime('%D')+'.')
            break
        
        #if not enough money to keep rising the stake, but enough to start over with the initial stake
        else:
            stake_current = stake_init
            
    plot_graph(list_date,list_bankroll)
    
    print(str_bankrupcy+'\nConclusion: \nYour bankroll would have gone from '+ str(bankroll_init) + '€ to '+"{:.2f}".format(round(list_bankroll[-1], 2))+
          '€, if you had bet on every '+league+' game on the favourite team, when its odds were '+str_category[odds_category]+
         '\nYour started with a '+str(stake_init)+'€ bet. Every time you lost a bet, you multiplied your last stake by '
         +str(multiplier)+'.')

    


    
box_layout = Layout(display='flex',
    flex_flow='row',
    border='solid 2px',
    align_items='flex-start',
    width='100%')

out= wg.interactive_output(main,{'bankroll_init':bankroll_init,'stake_init': stake_init, 'multiplier':multiplier,
                                   'odds_category':odds_category,'league':league})
right_box = wg.VBox([bankroll_init,stake_init,multiplier])
box = wg.HBox([league,odds_category,right_box],layout = box_layout)
display (box,out)

How does it work?

1. Choose the league you want to test this method with.
2. Each game we're betting on the favortie team. But as we've seen in the Excel file, not all odds categories have the same prediction accuracy. Choose one among the four.
3. Set your initial bankroll, and the amount of your initial stake. The multiplier will be used after a lost bet to determine your next stake. We're going to multiply automatically your last stake until you win, so you can cover your losses and be in profit.
You start over with the initial stake when you win a bet, or when you don't have enough in your bankroll to multiply your last stake after a loss. If you don't have enough to even make the first bet with the initial stake, you're out.


Output()